<a href="https://colab.research.google.com/github/edsonlcandido/gemini-cartola-rodada6/blob/main/rodada6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import requests
import json
url = "https://api.cartola.globo.com/atletas/mercado"
response = requests.get(url)
json_data = response.json()
data_dict = json.loads(response.text)
atletas_df = pd.json_normalize(data_dict["atletas"],max_level=2)
atletas_df=atletas_df.add_prefix('atleta_')
#print(atletas_df.columns.tolist())
atletas_df = atletas_df.rename(columns={'atleta_atleta_id':'atleta_id',
                                        'atleta_clube_id': 'clube_id',
                                        'atleta_posicao_id':'posicao_id',
                                        'atleta_status_id':'status_id'})

# Create the clubes DataFrame
clubes_df = pd.DataFrame.from_dict(data_dict["clubes"], orient='index')
clubes_df=clubes_df.add_prefix('clube_')
#print(clubes_df.columns.tolist())

# Create the posicoes DataFrame
posicoes_df = pd.DataFrame.from_dict(data_dict["posicoes"], orient='index')
posicoes_df=posicoes_df.add_prefix('posicao_')
#print(posicoes_df.columns.tolist())

#Create the status dataframe
status_df=pd.DataFrame.from_dict(data_dict["status"], orient='index')
status_df=status_df.add_prefix('status_')
#print(status_df.columns.tolist())

colunas = ['atleta_apelido','clube_abreviacao','posicao_abreviacao','status_nome', 'atleta_preco_num']

partidas_response = requests.get('https://api.cartola.globo.com/partidas')
partidas_json_data = partidas_response.json()
partidas_df = pd.json_normalize(partidas_json_data["partidas"],max_level=1)
#print(partidas_df.columns.tolist())
confrontos_df = partidas_df.merge(clubes_df.add_prefix('casa_'), left_on='clube_casa_id', right_on='casa_clube_id')
confrontos_df = confrontos_df.merge(clubes_df.add_prefix('visitante_'), left_on='clube_visitante_id', right_on='visitante_clube_id')
confrontos_df['confronto_slug'] = confrontos_df['casa_clube_abreviacao'] + ' X ' + confrontos_df['visitante_clube_abreviacao']
#print(confrontos_df.columns.tolist())
df = atletas_df.merge(posicoes_df, left_on='posicao_id',right_on='posicao_id')
df = df.merge(clubes_df, left_on='clube_id', right_on='clube_id')
df = df.merge(status_df, left_on='status_id', right_on='status_id')
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_casa_id'])
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_visitante_id'])
print(df.columns.tolist())


['atleta_minimo_para_valorizar', 'atleta_jogos_num', 'atleta_id', 'atleta_rodada_id', 'clube_id', 'posicao_id', 'status_id', 'atleta_pontos_num', 'atleta_media_num', 'atleta_variacao_num', 'atleta_preco_num', 'atleta_entrou_em_campo', 'atleta_slug', 'atleta_apelido', 'atleta_apelido_abreviado', 'atleta_nome', 'atleta_foto', 'atleta_scout.A', 'atleta_scout.CA', 'atleta_scout.DS', 'atleta_scout.FC', 'atleta_scout.FD', 'atleta_scout.FF', 'atleta_scout.FS', 'atleta_scout.G', 'atleta_scout.I', 'atleta_scout.SG', 'atleta_gato_mestre.scouts.media', 'atleta_gato_mestre.scouts.mandante', 'atleta_gato_mestre.scouts.visitante', 'atleta_gato_mestre.media_pontos_mandante', 'atleta_gato_mestre.media_pontos_visitante', 'atleta_gato_mestre.media_minutos_jogados', 'atleta_gato_mestre.minutos_jogados', 'atleta_scout.DE', 'atleta_scout.GS', 'atleta_scout.CV', 'atleta_scout.V', 'atleta_scout.FT', 'atleta_scout.PS', 'atleta_scout.PC', 'atleta_scout.DP', 'atleta_scout.GC', 'atleta_scout.PP', 'posicao_nome',

In [9]:
# prompt: gostaria de saber se o jogadores zagueiros no dataframe df pontuam mais [atleta_media_num] quando fazem desarmes [atleta_scout.DS] ou [atleta_scout.A] ou gols [atleta_scout.G]

df_zagueiros = df[df['posicao_abreviacao'] == 'ZAG']
df_zagueiros_com_gols = df_zagueiros[df_zagueiros['atleta_scout.G'] > 0]
df_zagueiros_com_desarmes = df_zagueiros[df_zagueiros['atleta_scout.DS'] > 0]
df_zagueiros_com_assistencias = df_zagueiros[df_zagueiros['atleta_scout.A'] > 0]

# Calculate average score for each category
media_gols = df_zagueiros_com_gols['atleta_media_num'].mean()
media_desarmes = df_zagueiros_com_desarmes['atleta_media_num'].mean()
media_assistencias = df_zagueiros_com_assistencias['atleta_media_num'].mean()

# Compare the averages
if media_gols > media_desarmes and media_gols > media_assistencias:
    print("Zagueiros pontuam mais quando fazem gols.")
elif media_desarmes > media_gols and media_desarmes > media_assistencias:
    print("Zagueiros pontuam mais quando fazem desarmes.")
else:
    print("Zagueiros pontuam mais quando fazem assistências.")

Zagueiros pontuam mais quando fazem assistências.
